In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v1_2 import CNN_v1_2

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
cnn = CNN_v1_2((480, 640, 3), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v1/weights/"

In [7]:
model_names = ["cnn_v1_2_1e_4_50",
          "cnn_v1_2_1e_5_30",
          "cnn_v1_2_1e_5_30",
          "cnn_v1_2_1e_3_4",
          "cnn_v1_2_1e_3_15",
          "cnn_v1_2_1e_3_25",
          "cnn_v1_2_1e_6_5",
          "cnn_v1_2_1e_6-02-0.95.hdf5",
        "cnn_v1_2_1e_6-next18-0.95.hdf5"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v1_2((480, 640, 3), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7047 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7063 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7060 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7057 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7067 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7033 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7034 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7044 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.7060 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v1/weights/cnn_v1_2_1e_4_50 behaved:
For cut-of-point 0.2 had accuracy 77.64944030391554:
[  2   5  -2   9  10   7  13  -5   4   5   6   5   7   5  -1   0  -2  -2
   4   2  -4  14  12  14   4   8  11  17   5  -1  12   0   6  16  12  20
  14  11  11   3   9  11  23   6  10   7   6   8   4  10  17  10   5   3
   6  10   1  -1   9  -4  13 -14   1   1  16   7  -4  -5  -7   7   7   4
   4   6   4   5   6   8   4  10   7   6   7  13  10   9   8   2  10   3
  -1  -2   5 -12   2  -2   8   0   6   2   7  -5   7  10  11  15  18   5
   0   7  13   8   8  12  12  11  16  13  15  10   6   8  15   7   2   6
   5   1   0   8   7   4   4  -1   8   2  17   7  11  20  10  -5   7   7
   0  -4   3  10  -4   7   6  -4   2   2   2   6  11   9  20  12   1   8
   4   6  -5   0  -2   5   3   3  -2  -1   8   2   1  -1   9  -4  -4   7
   5   8  12  17   5  -5  -4   2   4   1   5  -4   0   9   6  13   7  11
   6   3   9  19   1  10   9   1   1   0   0  -4   0  -9  -8  -5  -5  -4
   4   6  -1   3   5